In [3]:
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import models
import keras
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image

28x28 Grauwertpixel im Wertebereich von 0 bis 255
Anzahl Klassen = Anzahl Neuronen

In [5]:
def get_images(images_path):

    tensor_images = []

    train_or_test_folder = os.listdir(images_path)

    for folder in natsorted(train_or_test_folder):
        single_folder = os.path.join(images_path, folder)

        for file in os.listdir(single_folder):

            # get images as tensors
            filepath = os.path.join(single_folder, file)

            if filepath.lower().endswith(('.jpeg', '.jpg')):
                image = tf.io.read_file(filepath)
                tensor_image = tf.io.decode_image(image, channels=3, dtype=tf.dtypes.float32)
                tensor_image = tf.image.resize(tensor_image, [250, 250])
                tensor_images.append(tensor_image)
    
    return tensor_images


train_path_images = '../02_data_crop/train'
train_images = get_images(train_path_images)

test_path_images = '../02_data_crop//test'
test_images = get_images(test_path_images)

In [6]:
def get_label(images_path):

    tensor_label = []

    train_or_test_folder = os.listdir(images_path)

    for folder in natsorted(train_or_test_folder):
        single_folder = os.path.join(images_path, folder)

        for file in os.listdir(single_folder):
            tensor_label.append(int(single_folder))


    return tensor_label

train_path_labels = '../02_data_crop/train'
train_labels = get_images(train_path_labels)

test_path_labels = '../02_data_crop//test'
test_labels = get_images(test_path_labels)

In [ ]:
tf_dataset = tf.data.Dataset.from_tensor_slices((get_images(), get_label()))
full_dataset = tf_dataset.shuffle(20)

In [7]:
print('Shape Trainingsdaten: {}'.format(train_images.shape))
print('Dimension Bild Nr. 5: {}'.format(train_images[5].shape))
print('Label zu Bild Nr. 5: {}'.format(train_labels[5]))

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Laden und Vorverarbeiten der Daten
train_images = train_images.reshape(60000, 28, 28, 1)
train_images = train_images.astype('float32')
train_images /= 255

test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images.astype('float32')
test_images /= test_images

train_labels = to_categorial(train_labels)
test_labels = to_categorial(test_labels)

NrTrainimages = train_images.shape[0]
NrTestimages = test_images.shape[0]

In [ ]:
print('Trainingsdatensatz: {}'.format(train_images.shape))
print('Testdatensatz: {}'.format(test_images.shape))
print('Es sind {} Trainingsbilder und {} Testbilder vorhanden.'.format(NrTrainimages, NrTestimages))

In [ ]:
# Darstellung eines zufällig ausgewählten Bildes
randindex = random.randint(0, 60000)
plttitle = "Trainingsbild Nr. {} \n Klasse: {}".format(randindex, train_labels[randindex])
plt.imshow(train_images[randindex].reshape(28,28), cmap='gray')
plt.title(plttitle)
plt.axis('off')
plt.show()

In [2]:
# Netzwerkarchitektur
model = Sequential()

    # Detektion
model.add(Conv2D(32, kernel_size=(5,5), activation= 'relu', input_shape= mnist_inputshape))

    # Conv_Block 2
model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

    # Identifikation
model.add(Flatten())
model.add(Dense(128, activation='relu', name='features'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

NameError: name 'Sequential' is not defined

In [ ]:
# Festlegung der Verlustfunktion und Optimierung
model.compile(loss='categorial_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
# Trainig des Models
LOGDIR = "D:/logs"
tensorboard = TensorBoard(log_dir=LOGDIR, histogram_freq=0,
                             write_graph=True,
                             write_images=True)

# Hyperparameter
batch_size = 128
num_classes = 10
epochs = 12

history = model.fit(train_images, train_labels,
                    batch_size=batch_size,
                    callbacks=[tensorboard],
                    epochs=epochs,
                    verbose=1,
                    validation_data=(test_images, test_labels))

In [ ]:
# Evaluation
score = model.evaluate(test_images, test_labels)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

In [ ]:
# Speichern
model.save('Model.h5')

In [ ]:
' Laden des zu klassifizierenden Bildes'
path = ''
file = 'vogel.jpg'
img_file = os.path.join(path,file)

# Transformation in Bildgröße
img = image.load_img(img_file, target_size =(229,229))

x = image.img_to_array(img)
print(x.shape)
print(x)

In [ ]:
# zusätzliche Dimension für die Batchverarbeitung der Bilder
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)
print(x)

In [ ]:
# Klassifikation des Bildes
prediction = model.predict(x)

decoded = decode_predictions(prediction, top=3)[0]
print(decoded)

In [ ]:
# klassifiziertes Bild mit Klassenlegende
plt.figure(figsize=(8,8))

pos_x, pos_y = 1, 10
for id, lab, prob in decoded:
    plt.text(pos_x, pos_y, 'Klasse: {} - {:.2f}%'.format(lab, prob*100),
             fontsize=12, color='black',
             bbox=dict(boxstyle='round', pad=0.1, fc='white'))
    pos_y += 10